In [2]:
import numpy as np
from scipy.stats import t
from scipy.stats import norm
import matplotlib.pyplot as plt
from scipy import stats
import pandas as pd
import io
import requests
import seaborn as sns
sns.set_style('darkgrid')
from scipy.stats import pearsonr
import ssl
# следующая строчка подключает сертификат для защищенного соединения
ssl._create_default_https_context = ssl._create_unverified_context
%matplotlib inline

In [3]:
url='http://yustiks.ru/dataset/SPAM_text.csv'
data=pd.read_csv(url) 
data.tail()

,Category,Message
5567,spam,This is the 2nd time we have tried 2 contact u...
5568,ham,Will ü b going to esplanade fr home?
5569,ham,"Pity, * was in mood for that. So...any other s..."
5570,ham,The guy did some bitching but I acted like i'd...
5571,ham,Rofl. Its true to its name


In [4]:
print(data.loc[2, 'Message'])

Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's


In [5]:
# #Создадим новый атрибут Category, где будем указывать,
# что если данный текст является СПАМом, то 1, если не является, то 0
data["Category"] = [1 if each == "spam" else 0 for each in data["Category"]]
data.head()

,Category,Message
0,0,"Go until jurong point, crazy.. Available only ..."
1,0,Ok lar... Joking wif u oni...
2,1,Free entry in 2 a wkly comp to win FA Cup fina...
3,0,U dun say so early hor... U c already then say...
4,0,"Nah I don't think he goes to usf, he lives aro..."


In [6]:
# Создаем слова. На основе слов пишем для каждого текста словарь,
# где каждое слово - это ключ, а значение ключа - это сколько раз встречается данное слова в данном тексте.

# Как пример: рассмотрим 1 строку из датасета.

# Удалим все символы, не являющимися латинскими буквами
# Заглавные буквы меняем на строчные
# Разделим текст на слова
# В каждом слове выделяем корень слова
# Создаем список всех слов
import re
nlp_data = str(data.loc[2, 'Message'])
print(nlp_data)

Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's


In [7]:
nlp_data = re.sub("[^a-zA-Z]"," ", nlp_data)
print(nlp_data)

Free entry in   a wkly comp to win FA Cup final tkts   st May       Text FA to       to receive entry question std txt rate T C s apply            over   s


In [8]:
nlp_data = nlp_data.lower()
print(nlp_data)

free entry in   a wkly comp to win fa cup final tkts   st may       text fa to       to receive entry question std txt rate t c s apply            over   s


In [9]:
import nltk as nlp
nlp.download('punkt')
nlp_data = nlp.word_tokenize(nlp_data)
print(nlp_data)

['free', 'entry', 'in', 'a', 'wkly', 'comp', 'to', 'win', 'fa', 'cup', 'final', 'tkts', 'st', 'may', 'text', 'fa', 'to', 'to', 'receive', 'entry', 'question', 'std', 'txt', 'rate', 't', 'c', 's', 'apply', 'over', 's']


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Leo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [10]:
nlp.download('wordnet')
lemma = nlp.WordNetLemmatizer()
nlp_data = [lemma.lemmatize(word) for word in nlp_data]
print(nlp_data)

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Leo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


['free', 'entry', 'in', 'a', 'wkly', 'comp', 'to', 'win', 'fa', 'cup', 'final', 'tkts', 'st', 'may', 'text', 'fa', 'to', 'to', 'receive', 'entry', 'question', 'std', 'txt', 'rate', 't', 'c', 's', 'apply', 'over', 's']


In [11]:
nlp_data = " ".join(nlp_data)
description_list = []
for description in data["Message"]:
    description = re.sub("[^a-zA-Z]"," ",description)
    description = description.lower()
    description = nlp.word_tokenize(description)
    lemma = nlp.WordNetLemmatizer()
    description = [ lemma.lemmatize(word) for word in description]
    description = " ".join(description)
    description_list.append(description)

In [12]:
# Сколько всего получилось слов в словаре мешка слов
len(description_list)

5572

In [13]:
#Создаем bag-of-words, для этого выбираем 3000 максимально встречаемых слов
from sklearn.feature_extraction.text import CountVectorizer 
max_features = 3000
count_vectorizer = CountVectorizer(max_features = max_features, stop_words = "english")
sparce_matrix = count_vectorizer.fit_transform(description_list).toarray()
print("Самые часто встречаемые {} слов: {}".format(max_features,count_vectorizer.get_feature_names()))

Самые часто встречаемые 3000 слов: ['aah', 'aathi', 'abi', 'ability', 'abiola', 'able', 'absolutly', 'abt', 'abta', 'ac', 'academic', 'acc', 'accept', 'access', 'accident', 'accidentally', 'accordingly', 'account', 'ache', 'aco', 'action', 'activate', 'active', 'activity', 'actor', 'actual', 'actually', 'ad', 'add', 'addamsfa', 'added', 'addicted', 'addie', 'address', 'admin', 'admirer', 'adore', 'adoring', 'adult', 'advance', 'adventure', 'advice', 'advise', 'ae', 'affair', 'affection', 'afraid', 'aft', 'afternoon', 'aftr', 'agalla', 'age', 'ago', 'ah', 'aha', 'ahead', 'ahmad', 'aid', 'aight', 'ain', 'aint', 'air', 'airport', 'airtel', 'aiya', 'aiyah', 'aiyar', 'aiyo', 'aj', 'aka', 'al', 'alaipayuthe', 'album', 'alcohol', 'alert', 'alex', 'alfie', 'algarve', 'ali', 'alive', 'allah', 'allow', 'alright', 'alrite', 'alwys', 'amazing', 'american', 'amp', 'amt', 'amused', 'andros', 'angry', 'animation', 'annie', 'anniversary', 'announcement', 'anot', 'ansr', 'answer', 'answering', 'anti', 

In [31]:
y = data.iloc[:,0].values
x = sparce_matrix

In [32]:
#Делим данные на тренировочные и тестовые
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(x,y, test_size = 0.2, random_state = 42)

In [33]:
#наивный байес
from sklearn.naive_bayes import GaussianNB
nb = GaussianNB()
nb.fit(x_train,y_train)
print("the accuracy of our model: {}".format(nb.score(x_test,y_test)))

the accuracy of our model: 0.8780269058295964


In [34]:
from sklearn.metrics import classification_report
print(classification_report(y_test, nb.predict(x_test)))

              precision    recall  f1-score   support

           0       0.98      0.88      0.93       966
           1       0.53      0.89      0.66       149

    accuracy                           0.88      1115
   macro avg       0.75      0.88      0.79      1115
weighted avg       0.92      0.88      0.89      1115



In [35]:
#логистическая регрессия
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(max_iter = 200)
lr.fit(x_train,y_train)
print("our accuracy is: {}".format(lr.score(x_test,y_test)))

our accuracy is: 0.9811659192825112


In [36]:
print(classification_report(y_test, lr.predict(x_test)))

              precision    recall  f1-score   support

           0       0.98      1.00      0.99       966
           1       1.00      0.86      0.92       149

    accuracy                           0.98      1115
   macro avg       0.99      0.93      0.96      1115
weighted avg       0.98      0.98      0.98      1115



In [37]:
from sklearn.neighbors import KNeighborsClassifier
knn = KNeighborsClassifier(n_neighbors = 3)
knn.fit(x_train,y_train)
#print('Prediction: {}'.format(prediction))
print('With KNN (K=3) accuracy is: ',knn.score(x_test,y_test))

With KNN (K=3) accuracy is:  0.9417040358744395


In [38]:
print(classification_report(y_test, knn.predict(x_test)))

              precision    recall  f1-score   support

           0       0.94      1.00      0.97       966
           1       1.00      0.56      0.72       149

    accuracy                           0.94      1115
   macro avg       0.97      0.78      0.84      1115
weighted avg       0.95      0.94      0.93      1115



In [39]:
#Из всех выбранных моделей лучше всего дала результаты модель логистической регрессии.

In [ ]:
#Задача - определить класс, к которому относится тот или иной текст

In [40]:
#Удаляем слова, которые не имеют смысловой нагрузки (например, слова 'и', 'или', 'а' и другие)
from nltk.corpus import stopwords
from nltk import word_tokenize
import nltk
nltk.download('stopwords')
import string
import re
stop_words = set(stopwords.words('english'))

def remove_stopwords(line):
    word_tokens = word_tokenize(line)
    filtered_sentence = [w for w in word_tokens if not w in stop_words] 
    return " ".join(filtered_sentence)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Leo\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


In [41]:
# Далее функция, с помощью которой мы будем обрабатывать твиты

# переводим все слова в строчные буквы
# удаляем цифры
# удаляем пунктуацию
# удаляем стоп-слова
def preprocess(line):
  # все слова переводим в строчный текст
    line = line.lower()
  # удаляем цифры
    line = re.sub(r'\d+', '', line)
  # удаляем пунктуацию
    line = line.translate(line.maketrans("","", string.punctuation))
    line = remove_stopwords(line)
    return line

In [46]:
train = data

for i,line in enumerate(train.Message):
    train.Message[i] = preprocess(line)

<ipython-input-46-736ef504d7fa>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  train.Message[i] = preprocess(line)


In [47]:
train

,Category,Message
0,0,go jurong point crazy available bugis n great ...
1,0,ok lar joking wif u oni
2,1,free entry wkly comp win fa cup final tkts st ...
3,0,u dun say early hor u c already say
4,0,nah dont think goes usf lives around though
...,...,...
5567,1,nd time tried contact u u £ pound prize claim ...
5568,0,ü b going esplanade fr home
5569,0,pity mood soany suggestions
5570,0,guy bitching acted like id interested buying s...


In [51]:
#Разделим датасет на тренировочный и тестовый
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(train['Message'], train['Category'], 
                                                    test_size=0.5, stratify=train['Category'])

trainp=train[train.Category==1]
trainn=train[train.Category==0]
print(trainp.info())
trainn.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 747 entries, 2 to 5567
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  747 non-null    int64 
 1   Message   747 non-null    object
dtypes: int64(1), object(1)
memory usage: 17.5+ KB
None
<class 'pandas.core.frame.DataFrame'>
Int64Index: 4825 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   Category  4825 non-null   int64 
 1   Message   4825 non-null   object
dtypes: int64(1), object(1)
memory usage: 113.1+ KB


In [52]:
# Можно заметить, что классы несбалансированы: в классе 1 2242 элемента, а в классе 0 их 29720.

# Создадим bag-of-words вектора для всех твитов
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
vect = CountVectorizer()
tf_train=vect.fit_transform(X_train)  #train the vectorizer, build the vocablury
tf_test=vect.transform(X_test)  #get same encodings on test data as of vocabulary built

In [53]:
#наивный байес
from sklearn.naive_bayes import MultinomialNB
model = MultinomialNB()
model.fit(X=tf_train,y=y_train)
expected = y_test
predicted=model.predict(tf_test)
from sklearn import metrics

print(metrics.classification_report(expected, predicted))
print(metrics.confusion_matrix(expected, predicted))

              precision    recall  f1-score   support

           0       0.99      0.99      0.99      2413
           1       0.92      0.91      0.91       373

    accuracy                           0.98      2786
   macro avg       0.95      0.95      0.95      2786
weighted avg       0.98      0.98      0.98      2786

[[2382   31]
 [  35  338]]


In [ ]:
# Можно заметить, что класс 1 предсказывается намного хуже, чем класс 0: класса 1 намного меньше по числу элементов, чем класс 0.

# Сбалансируем датасет

In [55]:
train_imbalanced = train
from sklearn.utils import resample
df_majority = train[train.Category==0]
df_minority = train[train.Category==1]
 
# Upsample minority class
df_minority_upsampled = resample(df_minority, 
                                 replace=True,     # sample with replacement
                                 n_samples=len(df_majority),    # to match majority class
                                 random_state=123) # reproducible results
 
# Combine majority class with upsampled minority class
df_upsampled = pd.concat([df_majority, df_minority_upsampled])
 
# Display new class counts
print("Before")
print(train.Category.value_counts())
print("After")
print(df_upsampled.Category.value_counts())

X_train, X_test, y_train, y_test = train_test_split(df_upsampled['Message'], df_upsampled['Category'], test_size=0.5, stratify=df_upsampled['Category'])

Before
0    4825
1     747
Name: Category, dtype: int64
After
1    4825
0    4825
Name: Category, dtype: int64


In [58]:
#тренировочных данных стало больше, и классы уравнялись
#балансировка привела к улучшению результата

In [57]:
tf_train=vect.transform(X_train)
tf_test=vect.transform(X_test)
model.fit(X=tf_train,y=y_train)
expected = y_test
predicted=model.predict(tf_test)
from sklearn import metrics

print(metrics.classification_report(expected, predicted))
print('Матрица confusion')
print(metrics.confusion_matrix(expected, predicted))

              precision    recall  f1-score   support

           0       0.97      0.96      0.96      2412
           1       0.96      0.97      0.96      2413

    accuracy                           0.96      4825
   macro avg       0.96      0.96      0.96      4825
weighted avg       0.96      0.96      0.96      4825

Матрица confusion
[[2306  106]
 [  73 2340]]
